# OC PROJET 5 - AUTOMATICALLY CATEGORIZE QUESTIONS
#### CLEANING AND ANALYSIS NOTEBOOK
<br></br>
### SOMMAIRE
- <a href="#C1">I. Nettoyage des données</a>
    
- <a href="#C2">II. Feature Engineering</a>
    
- <a href="#C3">III. Exploration des données</a>
    - 1. Matrice des corrélations
    - 2. Analyse temporelle
    - 3. Analyse Quanti/Quanti
    - 4. Analyse Quanti/Quali
    - 5. Analyse Quali/Quali
    - 6. ACP

<font size="5">1. Importation des librairies</font>

In [90]:
# importation des librairies
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import font_manager as rcParams
import matplotlib.patheffects as path_effects
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
from scipy.stats import pearsonr
from scipy.stats import f_oneway
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

<font size="5">2. Paramétrages Data Visualisation</font>

In [91]:
# Ajouter une ombre à la police
shadow = path_effects.withSimplePatchShadow(offset = (1, - 0.75), 
shadow_rgbFace = 'darkblue', alpha = 0.25)

# changer la police dans les graphiques, les couleurs 
# et augmenter la résolution d'affichage
plt.rcParams['font.family'] = 'Ebrima'
plt.rcParams['text.color'] = 'white'
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 200
plt.style.use('dark_background')

# set le theme seaborn
sns.set_style('darkgrid', {'axes.facecolor': '0.2',
'text.color': 'white', 'figure.figsize': (20, 16)})
plt.rcParams['figure.facecolor'] = '0.2'

# suppression de l'affichage max des colonnes
pd.set_option('display.max_columns', None)

### SQL REQUEST CODE 

SELECT TOP 10000000 Title, Body, Tags, Id, Score, ViewCount, AnswerCount,
CreationDate, LastActivityDate, CommentCount

FROM Posts 

WHERE PostTypeId = 1 AND ViewCount > 100 AND Score > 3 AND AnswerCount > 0 
AND LEN(Tags) - LEN(REPLACE(Tags, '<','')) >= 5 AND CommentCount > 0

# <a name="C2">I. Nettoyage des données</a>

In [101]:
df = pd.read_csv('QueryResults.csv')

In [93]:
df

,Title,Body,Tags,Id,Score,ViewCount,AnswerCount,CreationDate,LastActivityDate,CommentCount
0,"JQuery - AJAX dialog modal, can't hit enter key to submit form","<p>On a website I'm working on, when you click sign on, a jquery dialoge modal pops up, but you have to click on OK to submit the form, you can't just hit enter. I need it to be able to have enter work also. It seems like what I have should work, but it doesn't</p>\n\n<p>I'm using jquery-1.3.2.js. I also have a php file with the following piece of code in it: `</p>\n\n<pre><code> &lt;tr valign=""top"" align=""right"" style=""height:40px""&gt;&lt;td &gt;\n\n &lt;div id=""signin""&gt;\n\n &lt;table style=""margin-top:4px;margin-right:4px;border-style:solid;border-width:1px""&gt;\n\n &lt;tr&gt;&lt;td style=""width:165px;""&gt; \n\n &lt;div&gt;&lt;center&gt;\n\n &lt;a title=""Sign In"" onclick=""LoginDialogOpen()"" href=""javascript:void();""&gt;Sign In&lt;/a&gt;&lt;b&gt;&amp;nbsp;&amp;nbsp; | &amp;nbsp;&amp;nbsp;&lt;/b&gt;\n\n &lt;a title=""Create Account"" href=""CreateAccount.html""&gt;Create Account&lt;/a&gt;\n\n &lt;/center&gt;&lt;/div&gt; \n\n &lt;/td&gt;&lt;/tr&gt;\n\n &lt;/table&gt;\n\n &lt;/div&gt;\n\n &lt;/td&gt;&lt;/tr&gt;\n</code></pre>\n\n<p></p>\n\n<pre><code> &lt;div id=""Signin_Dialog"" &gt;\n\n &lt;div id=""bg""&gt;\n\n &lt;label&gt;&lt;span&gt;Email:&lt;/span&gt;&lt;/label&gt;\n\n &lt;input type=""text"" name=""email"" id=""email"" class=""dialog-input-text""/&gt;\n\n &lt;br&gt;\n\n\n\n &lt;label&gt;&lt;span&gt;Password:&lt;/span&gt;&lt;/label&gt;\n\n &lt;input type=""password"" name=""password"" id=""password"" class=""dialog-input-text""/&gt;\n\n &lt;br&gt;\n\n &lt;br&gt;\n\n &lt;center&gt;&lt;b&gt;&lt;label id=""login_error"" style=""color:red""&gt;&lt;span&gt;&amp;nbsp;&lt;/span&gt;&lt;/label&gt;&lt;/center&gt;&lt;/b&gt;\n\n\n\n &lt;/div&gt;\n\n&lt;/div&gt;\n\n\n\n&lt;script&gt;\n\n $('#login_dialog').dialog({\n\n autoOpen: false,\n\n width: 310,\n\n overlay: { opacity: 0.5, background: ""black"" },\n\n modal: true,\n\n buttons: {\n\n ""Ok"": function() { \n\n $(""body"").addClass(""curWait""); \n\n sql = ""select client_id from users where email = '"" + $(""#email"")[0].value + ""' and login_password='"" + $(""#password"")[0].value + ""'"";\n\n $.get('BongoData.php', { task:""SQLResultToJSON"", sql: sql}, ResultOfLoginAttempt, ""json"");\n\n }, \n\n ""Cancel"": function() { \n\n $(this).dialog(""close""); \n\n } \n\n }\n\n });\n\n\n&lt;/script&gt;`\n</code></pre>\n\n<p>i have a javascript file with the following function:</p>\n\n<pre><code>function LoginDialogOpen(){\n\n $('#login_dialog').dialog('open');\n $('#login_dialog').keypress(function(e) {\n if (e.which == 13) {\n $(""body"").addClass(""curWait""); \n\n sql = ""select client_id from users where email = '"" + $(""#email"")[0].value + ""' and login_password='"" + $(""#password"")[0].value + ""'"";\n\n $.get('BongoData.php', { task:""SQLResultToJSON"", sql: sql}, ResultOfLoginAttempt, ""json"");\n }\n});\n</code></pre>\n\n<p>}</p>\n\n<p>That is the code I have, I don't understand why it isn't working. </p>\n\n<p>I also had it try $('#login_dialog').dialog('isOpen'); right after i opened it, but it always returned false oddly enough. Please help if you can.</p>\n",<javascript><jquery><ajax><jquery-ui><enter>,1113203,4,7735,3,2009-07-11 08:03:25,2010-09-28 09:27:26,3
1,How do I do large non-blocking updates in PostgreSQL?,"<p>I want to do a large update on a table in PostgreSQL, but I don't need the transactional integrity to be maintained across the entire operation, because I know that the column I'm changing is not going to be written to or read during the update. I want to know if there is an easy way <em>in the psql console</em> to make these types of operations faster. </p>\n\n<p>For example, let's say I have a table called ""orders"" with 35 million rows, and I want to do this: </p>\n\n<pre><code>UPDATE orders SET status = null;\n</code></pre>\n\n<p>To avoid being diverted to an offtopic dis

In [94]:
df.isnull().sum()

Title               0
Body                0
Tags                0
Id                  0
Score               0
ViewCount           0
AnswerCount         0
CreationDate        0
LastActivityDate    0
CommentCount        0
dtype: int64

In [95]:
df.loc[df.duplicated(keep = False),:]

,Title,Body,Tags,Id,Score,ViewCount,AnswerCount,CreationDate,LastActivityDate,CommentCount


In [96]:
df.dtypes

Title               object
Body                object
Tags                object
Id                   int64
Score                int64
ViewCount            int64
AnswerCount          int64
CreationDate        object
LastActivityDate    object
CommentCount         int64
dtype: object

In [97]:
df.describe()

,Id,Score,ViewCount,AnswerCount,CommentCount
count,5.000000e+04,50000.000000,5.000000e+04,50000.000000,50000.000000
mean,2.620070e+07,28.042520,2.683093e+04,3.398660,3.688680
std,1.930342e+07,194.846305,1.397911e+05,4.026946,3.349398
min,4.000000e+00,4.000000,1.010000e+02,1.000000,1.000000
25%,9.724780e+06,5.000000,1.945000e+03,1.000000,1.000000
50%,2.299686e+07,7.000000,5.350000e+03,2.000000,3.000000
75%,3.885722e+07,14.000000,1.547175e+04,4.000000,5.000000
max,7.634388e+07,25632.000000,1.276011e+07,134.000000,51.000000


In [100]:
# Fonction pour enlever la balise <p>
def remove_html_tags(text):
    clean = re.compile('<.*?>|\n')
    return re.sub(clean, '', text)

In [102]:
def preprocess_text(text):
    # Supprimer les balises HTML
    text = re.sub('<.*?>', '', text)
    
    # Convertir en minuscules
    text = text.lower()
    
    # Supprimer la ponctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Supprimer les mots vides (stop words)
    stop_words = set(stopwords.words('english'))
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatisation des mots
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Rejoindre les tokens prétraités en une seule chaîne de texte
    processed_text = ' '.join(tokens)
    
    return processed_text

In [103]:
df['Body'] = df['Body'].apply(preprocess_text)

In [83]:
df['Body'].head()

0    website im working click sign jquery dialoge modal pop click ok submit form cant hit enter need able enter work also seems like work doesnt im using jquery132js also php file following piece code lttr valigntop alignright styleheight40pxgtlttd gt ltdiv idsigningt lttable stylemargintop4pxmarginright4pxborderstylesolidborderwidth1pxgt lttrgtlttd stylewidth165pxgt ltdivgtltcentergt lta titlesign onclicklogindialogopen hrefjavascriptvoidgtsign inltagtltbgtampnbspampnbsp ampnbspampnbspltbgt lta titlecreate account hrefcreateaccounthtmlgtcreate accountltagt ltcentergtltdivgt lttdgtlttrgt lttablegt ltdivgt lttdgtlttrgt ltdiv idsignin_dialog gt ltdiv idbggt ltlabelgtltspangtemailltspangtltlabelgt ltinput typetext nameemail idemail classdialoginputtextgt ltbrgt ltlabelgtltspangtpasswordltspangtltlabelgt ltinput typepassword namepassword idpassword classdialoginputtextgt ltbrgt ltbrgt ltcentergtltbgtltlabel idlogin_error stylecolorredgtltspangtampnbspltspangtltlabelgtltcentergtltbgt ltdivg

In [71]:
# Ajuster le nombre maximum de caractères affichés
pd.set_option('display.max_colwidth', None)

In [72]:
df['Body'].head()

0    website im working click sign jquery dialoge modal pop click ok submit form cant hit enter need able enter work also seems like work doesnt im using jquery132js also php file following piece code lttr valigntop alignright styleheight40pxgtlttd gt ltdiv idsigningt lttable stylemargintop4pxmarginright4pxborderstylesolidborderwidth1pxgt lttrgtlttd stylewidth165pxgt ltdivgtltcentergt lta titlesign onclicklogindialogopen hrefjavascriptvoidgtsign inltagtltbgtampnbspampnbsp ampnbspampnbspltbgt lta titlecreate account hrefcreateaccounthtmlgtcreate accountltagt ltcentergtltdivgt lttdgtlttrgt lttablegt ltdivgt lttdgtlttrgt ltdiv idsignin_dialog gt ltdiv idbggt ltlabelgtltspangtemailltspangtltlabelgt ltinput typetext nameemail idemail classdialoginputtextgt ltbrgt ltlabelgtltspangtpasswordltspangtltlabelgt ltinput typepassword namepassword idpassword classdialoginputtextgt ltbrgt ltbrgt ltcentergtltbgtltlabel idlogin_error stylecolorredgtltspangtampnbspltspangtltlabelgtltcentergtltbgt ltdivg

In [51]:
df

,Title,Body,Tags,Id,Score,ViewCount,AnswerCount,CreationDate,LastActivityDate,CommentCount,ActivityTime
0,"JQuery - AJAX dialog modal, can't hit enter key to submit form","On a website I'm working on, when you click sign on, a jquery dialoge modal pops up, but you hav...",[<javascript><jquery><ajax><jquery-ui><enter>],1113203,4,7735,3,2009-07-11 08:03:25,2010-09-28 09:27:26,3,444
1,How do I do large non-blocking updates in PostgreSQL?,"I want to do a large update on a table in PostgreSQL, but I don't need the transactional integri...",[<postgresql><transactions><sql-update><plpgsql><dblink>],1113277,83,53458,9,2009-07-11 08:46:42,2023-05-05 05:12:05,4,5045
2,Manually setting session ID in Express,I have an Angular/Express app and am trying to implement some kind of restful auth. The Express ...,[<javascript><angularjs><node.js><express><passport.js>],20060800,4,3606,1,2013-11-19 00:27:34,2015-01-08 16:58:59,2,415
3,Include Google Maps API Key in open source project?,Is it okay to put your Google Maps API Key into your source code and publish it?Others could tak...,[<security><api><open-source><google-maps><publish>],1113292,10,1676,1,2009-07-11 09:01:50,2009-07-11 13:19:49,1,0
4,Emacs ido-style shell,Is there a command line shell or shell customization that supports emacs-style ido find file? I...,[<bash><shell><emacs><eshell><ido>],1112805,20,2683,7,2009-07-11 02:51:56,2020-01-14 13:00:16,5,3839
...,...,...,...,...,...,...,...,...,...,...,...
49995,Linq filtering an IQueryable<T> (System.Data.Linq.DataQuery) object by a List<T> (System.Collect...,My IQueryable line is: // find all timesheets for this period - from db so System.Data.Linq.Data...,[<c#><asp.net-mvc><linq-to-sql><list><iqueryable>],2666065,5,6390,2,2010-04-19 08:21:35,2010-04-19 08:47:48,1,0
49996,How can I gzip my JavaScript and CSS files?,"I have a problem, I have to gzip a prototype Lib, but i totaly have no idea how to do this, wher...",[<javascript><css><apache><http><gzip>],2666120,23,50791,6,2010-04-19 08:33:01,2015-10-05 06:22:59,1,1994
49997,how to know location of return address on stack c/c++,i have been reading about a function that can overwrite its return address.void foo(const char* ...,[<c++><c><winapi><x86><stack>],2666301,5,7400,1,2010-04-19 09:05:09,2010-04-19 10:01:46,2,0
49998,Is it possible to use Firebase Cloud Messaging in iOS app without Apple Developer Program?,I wonder if it is possible to use firebase cloud messaging with iOS app without Apple Developer ...,[<ios><swift><firebase><apple-push-notifications><firebase-cloud-messaging>],40194149,5,1744,1,2016-10-22 16:00:06,2016-12-30 09:19:53,2,68


# <a name="C2">II. Feature Engineering</a>

In [64]:
df

,Title,Body,Tags,Id,Score,ViewCount,AnswerCount,CreationDate,LastActivityDate,CommentCount,nb_of_tags
0,"JQuery - AJAX dialog modal, can't hit enter key to submit form","On a website I'm working on, when you click sign on, a jquery dialoge modal pops up, but you hav...",<javascript><jquery><ajax><jquery-ui><enter>,1113203,4,7735,3,2009-07-11 08:03:25,2010-09-28 09:27:26,3,5
1,How do I do large non-blocking updates in PostgreSQL?,"I want to do a large update on a table in PostgreSQL, but I don't need the transactional integri...",<postgresql><transactions><sql-update><plpgsql><dblink>,1113277,83,53458,9,2009-07-11 08:46:42,2023-05-05 05:12:05,4,5
2,Manually setting session ID in Express,I have an Angular/Express app and am trying to implement some kind of restful auth. The Express ...,<javascript><angularjs><node.js><express><passport.js>,20060800,4,3606,1,2013-11-19 00:27:34,2015-01-08 16:58:59,2,5
3,Include Google Maps API Key in open source project?,Is it okay to put your Google Maps API Key into your source code and publish it?Others could tak...,<security><api><open-source><google-maps><publish>,1113292,10,1676,1,2009-07-11 09:01:50,2009-07-11 13:19:49,1,5
4,Emacs ido-style shell,Is there a command line shell or shell customization that supports emacs-style ido find file? I...,<bash><shell><emacs><eshell><ido>,1112805,20,2683,7,2009-07-11 02:51:56,2020-01-14 13:00:16,5,5
...,...,...,...,...,...,...,...,...,...,...,...
49995,Linq filtering an IQueryable<T> (System.Data.Linq.DataQuery) object by a List<T> (System.Collect...,My IQueryable line is: // find all timesheets for this period - from db so System.Data.Linq.Data...,<c#><asp.net-mvc><linq-to-sql><list><iqueryable>,2666065,5,6390,2,2010-04-19 08:21:35,2010-04-19 08:47:48,1,5
49996,How can I gzip my JavaScript and CSS files?,"I have a problem, I have to gzip a prototype Lib, but i totaly have no idea how to do this, wher...",<javascript><css><apache><http><gzip>,2666120,23,50791,6,2010-04-19 08:33:01,2015-10-05 06:22:59,1,5
49997,how to know location of return address on stack c/c++,i have been reading about a function that can overwrite its return address.void foo(const char* ...,<c++><c><winapi><x86><stack>,2666301,5,7400,1,2010-04-19 09:05:09,2010-04-19 10:01:46,2,5
49998,Is it possible to use Firebase Cloud Messaging in iOS app without Apple Developer Program?,I wonder if it is possible to use firebase cloud messaging with iOS app without Apple Developer ...,<ios><swift><firebase><apple-push-notifications><firebase-cloud-messaging>,40194149,5,1744,1,2016-10-22 16:00:06,2016-12-30 09:19:53,2,5


In [104]:
# Convertir les variables 'CreationDate' et 'LastActivityDate' en format de date
df['CreationDate'] = pd.to_datetime(df['CreationDate'])
df['LastActivityDate'] = pd.to_datetime(df['LastActivityDate'])

# Calculer la différence en jours entre les deux variables
df['ActivityTime'] = (df['LastActivityDate'] - df['CreationDate']).dt.days

In [49]:
df

,Title,Body,Tags,Id,Score,ViewCount,AnswerCount,CreationDate,LastActivityDate,CommentCount,ActivityTime
0,"JQuery - AJAX dialog modal, can't hit enter key to submit form","On a website I'm working on, when you click sign on, a jquery dialoge modal pops up, but you hav...",<javascript><jquery><ajax><jquery-ui><enter>,1113203,4,7735,3,2009-07-11 08:03:25,2010-09-28 09:27:26,3,444
1,How do I do large non-blocking updates in PostgreSQL?,"I want to do a large update on a table in PostgreSQL, but I don't need the transactional integri...",<postgresql><transactions><sql-update><plpgsql><dblink>,1113277,83,53458,9,2009-07-11 08:46:42,2023-05-05 05:12:05,4,5045
2,Manually setting session ID in Express,I have an Angular/Express app and am trying to implement some kind of restful auth. The Express ...,<javascript><angularjs><node.js><express><passport.js>,20060800,4,3606,1,2013-11-19 00:27:34,2015-01-08 16:58:59,2,415
3,Include Google Maps API Key in open source project?,Is it okay to put your Google Maps API Key into your source code and publish it?Others could tak...,<security><api><open-source><google-maps><publish>,1113292,10,1676,1,2009-07-11 09:01:50,2009-07-11 13:19:49,1,0
4,Emacs ido-style shell,Is there a command line shell or shell customization that supports emacs-style ido find file? I...,<bash><shell><emacs><eshell><ido>,1112805,20,2683,7,2009-07-11 02:51:56,2020-01-14 13:00:16,5,3839
...,...,...,...,...,...,...,...,...,...,...,...
49995,Linq filtering an IQueryable<T> (System.Data.Linq.DataQuery) object by a List<T> (System.Collect...,My IQueryable line is: // find all timesheets for this period - from db so System.Data.Linq.Data...,<c#><asp.net-mvc><linq-to-sql><list><iqueryable>,2666065,5,6390,2,2010-04-19 08:21:35,2010-04-19 08:47:48,1,0
49996,How can I gzip my JavaScript and CSS files?,"I have a problem, I have to gzip a prototype Lib, but i totaly have no idea how to do this, wher...",<javascript><css><apache><http><gzip>,2666120,23,50791,6,2010-04-19 08:33:01,2015-10-05 06:22:59,1,1994
49997,how to know location of return address on stack c/c++,i have been reading about a function that can overwrite its return address.void foo(const char* ...,<c++><c><winapi><x86><stack>,2666301,5,7400,1,2010-04-19 09:05:09,2010-04-19 10:01:46,2,0
49998,Is it possible to use Firebase Cloud Messaging in iOS app without Apple Developer Program?,I wonder if it is possible to use firebase cloud messaging with iOS app without Apple Developer ...,<ios><swift><firebase><apple-push-notifications><firebase-cloud-messaging>,40194149,5,1744,1,2016-10-22 16:00:06,2016-12-30 09:19:53,2,68


# <a name="C2">III. Feature Extraction</a>

In [108]:
# Fonction pour extraire les tags d'une chaîne de caractères
def extract_tags(tag_string):
    tags = re.findall(r'<(.*?)>', tag_string)
    return tags

# Application de la fonction d'extraction des tags à la colonne 'tags' du DataFrame
df['Tags'] = df['Tags'].apply(extract_tags)

# Extraction des tags uniques à partir de toutes les lignes
unique_tags = set([tag for tags_list in df['Tags'] for tag in tags_list])

# Affichage des tags uniques
print("Tags uniques :")
for tag in unique_tags:
    print(tag)

Tags uniques :
rosetta-2
ntpd
shake
microxml
vectorization
invalidation
undo-redo
keydown
scala-ide
directory-tree
clang-static-analyzer
gnu-screen
modin
uv-mapping
arm
distribution
readdirectorychangesw
cursor-position
single-quotes
delimiter
tizen-web-app
logfiles
xcode10
image-segmentation
apacheds
offloading
query-builder
colorfilter
agora.io
toggle
face-id
cross-join
oauth2client
phpbb3
decidable
playframework-evolutions
zend-form
stockquotes
autodoc
inlines
chromakey
dmd
development-environment
mediator
client-certificates
geoip
tilde-expansion
file-manipulation
structuremap3
cruisecontrol
arrows
pyzmq
zero-copy
codedom
member-access
buildout
android-backup-service
rapydscript
git-history
geckoview
nstoolbar
python-embedding
loops
reporting-services
vue-props
textfield
sigterm
model-view-controller
sql-server-ce
mailchimp-api-v3.0
zk
forward-declaration
atlassian-plugin-sdk
daemon
bytestream
acfpro
newsletter
url-parsing
mobile-development
entity-framework-extended
onclick
jscrip

qsortfilterproxymodel
badge
digit-separator
wcf-client
storage
winmain
oggvorbis
ios-app-group
type-erasure
wp-nav-walker
resteasy
emma
grayscale
gitkraken
sharppcap
downgrade
shinyapps
uiinterfaceorientation
sparql
form-helpers
sketch-3
bouncycastle
call-graph
python-3.6
unmanaged-memory
chakra-ui
replace
quartz-scheduler
prng
filestream
hlist
c++builder-xe2
jquery-traversing
xmlreader
browser-feature-detection
spring-messaging
edismax
strdup
asp.net-development-serv
ebnf
azure-signalr
full-trust
cancancan
uploaddataasync
boost-interprocess
deprecation-warning
vps
cross-product
pdf-generation
gwt
xcode7.2
html5-draggable
better-sqlite3
wcf-binding
desktop-application
textview
tesseract.js
spacy-3
remoteobject
unnotificationserviceextension
jeditorpane
fillna
saucelabs
fonts
simd
data-integrity
text
uno
ios-enterprise
wikipedia-api
memoization
swift-extensions
playframework-2.1
xcache
gnu-toolchain
mvvm-light
target-platform
ruby-on-rails-3
jquery-ui-autocomplete
dependent-type
react-d

concatenation
nse
xamarin.auth
dinktopdf
google-oauth-java-client
facebook-sdk-4.0
gspread
custom-error-handling
superuser
buildconfig
qcombobox
intel-pin
dicom
azure-sphere
accounts
visual-studio-2003
ctrl
qmake
curses
tail
schema
imageflow
express-jwt
ora-01722
complexity-theory
realm
rtos
python-3.10
scala-2.8
ucs2
logarithm
winzip
gcov
xcode-build-phase
hibernate-annotations
datomic
nexus-s
gitlab-pipelines
hudson
response.redirect
foaf
gulp-sourcemaps
microphone
jcmd
subscribe
actionpack
aframe
nexus-one
onload-event
jdk6
ng-map
read-uncommitted
charset
service-locator
system.web
continuous-testing
masonry
ng-messages
cpack
edge-detection
koin
android-contentresolver
slidingdrawer
tlbinf32
greendao
pusher
point-cloud-library
jsse
geofire
tsconfig
amazon-athena
archiva
delphi
vue-cli-3
capslock
viewaction
glfw
eventsource
eloquent
cp
python-c-extension
msal.js
global
p2p
aws-codebuild
motion
ts-jest
qquickwidget
datacontext
maven-publish
iso8601
ruby-2.7
function-call
trial
app-tra

xmlwriter
scikit-image
microsoft-teams
preloading
operating-system
opc-ua
static-resource
google-closure
monetdb
code-signing-entitlements
ecmascript-2016
srand
simplex-noise
throw
iis-10
preact
encog
horizontalscrollview
mysql-variables
primeng-calendar
minimax
advanced-custom-fields
redux-observable
zombie.js
launch-services
sctp
servicemanager
godi
abap
source-code-protection
gdata
dotnet-cli
karabiner
js-of-ocaml
nimbus
lob
product
iframe
tcc
sortedset
glow
webpack-dev-server
kafka-producer-api
h.265
android-jetpack-compose-list
cylindrical
powershell-2.0
feathersjs
ftp-server
operator-overloading
sticky
yii2
mapping
logcat
nsnumberformatter
python-collections
fpc
google-sheets-query
jetpack-compose-navigation
http-status-code-412
apache-commons-logging
prediction
dynamics-crm-2013
edi
yaws
django-settings
pos-tagger
logstash-jdbc
bulkupdate
google-api-dotnet-client
azure-aks
mcafee
in-app-update
computation-expression
dhcp
azure-devops-hosted-agent
nsviewcontroller
haskell-polysem

libmemcached
shared-ptr
okhttp
socks
ilog
acceptance-testing
urn
facebook-messenger-bot
broom
getsockopt
splint
azure-eventgrid
collectionviewsource
react-native-code-push
web-controls
weasyprint
utf-16
computed-style
upnp
stderr
controlvalueaccessor
for-in-loop
public-key-encryption
ess
itemcontainerstyle
openfst
model-binding
b-tree
datacontractserializer
cross-application
polling
object-detection
rank
php-7
istio
mail-form
udp
name-lookup
busybox
duplicate-data
dtreeviz
namespace-package
npm-build
onvif
rcpp11
case-when
drupal-render
miniprofiler
docker-daemon
ospf
mv
diskimage
firebird
fish
iequatable
zappa
oniguruma
extjs6-classic
folium
doctrine-mapping
w3-total-cache
7zip
communicate
mpfr
r-rownames
permanent-generation
lodash
idispatch
image-caching
mvccontrib-testhelper
numerical-integration
reactor
shared-memory
proxy-classes
sms-gateway
jboss-logging
property-graph
novacode-docx
default.png
idfa
qgraphicsscene
exponential
automator
django-tinymce
gatsby-image
truthtable
coll

In [109]:
df

,Title,Body,Tags,Id,Score,ViewCount,AnswerCount,CreationDate,LastActivityDate,CommentCount,ActivityTime
0,"JQuery - AJAX dialog modal, can't hit enter key to submit form",website im working click sign jquery dialoge modal pop click ok submit form cant hit enter need able enter work also seems like work doesnt im using jquery132js also php file following piece code lttr valigntop alignright styleheight40pxgtlttd gt ltdiv idsigningt lttable stylemargintop4pxmarginright4pxborderstylesolidborderwidth1pxgt lttrgtlttd stylewidth165pxgt ltdivgtltcentergt lta titlesign onclicklogindialogopen hrefjavascriptvoidgtsign inltagtltbgtampnbspampnbsp ampnbspampnbspltbgt lta titlecreate account hrefcreateaccounthtmlgtcreate accountltagt ltcentergtltdivgt lttdgtlttrgt lttablegt ltdivgt lttdgtlttrgt ltdiv idsignin_dialog gt ltdiv idbggt ltlabelgtltspangtemailltspangtltlabelgt ltinput typetext nameemail idemail classdialoginputtextgt ltbrgt ltlabelgtltspangtpasswordltspangtltlabelgt ltinput typepassword namepassword idpassword classdialoginputtextgt ltbrgt ltbrgt ltcentergtltbgtltlabel idlogin_error stylecolorredgtltspangtampnbspltspangtltlabelgtltcentergtltbgt ltdivgt ltdivgt ltscriptgt login_dialogdialog autoopen false width 310 overlay opacity 05 background black modal true button ok function bodyaddclasscurwait sql select client_id user email email0value login_password password0value getbongodataphp tasksqlresulttojson sql sql resultofloginattempt json cancel function thisdialogclose ltscriptgt javascript file following function function logindialogopen login_dialogdialogopen login_dialogkeypressfunctione ewhich 13 bodyaddclasscurwait sql select client_id user email email0value login_password password0value getbongodataphp tasksqlresulttojson sql sql resultofloginattempt json code dont understand isnt working also try login_dialogdialogisopen right opened always returned false oddly enough please help,"[javascript, jquery, ajax, jquery-ui, enter]",1113203,4,7735,3,2009-07-11 08:03:25,2010-09-28 09:27:26,3,444
1,How do I do large non-blocking updates in PostgreSQL?,want large update table postgresql dont need transactional integrity maintained across entire operation know column im changing going written read update want know easy way psql console make type operation faster example let say table called order 35 million row want update order set status null avoid diverted offtopic discussion let assume value status 35 million column currently set nonnull value thus rendering index useless problem statement take long time go effect solely locking changed row locked entire update complete update might take 5 hour whereas something like update order set status null order_id gt 0 order_id lt 1000000 might take 1 minute 35 million row breaking chunk 35 would take 35 minute save 4 hour 25 minute could break even script using pseudocode 0 3500 db_operation update order set status null order_id gt i1000 order_id lt i11000 operation might complete minute rather 35 come im really asking dont want write freaking script break operation every single time want big onetime update like way accomplish want entirely within sql,"[postgresql, transactions, sql-update, plpgsql, dblink]",1113277,83,53458,9,2009-07-11 08:46:42,2023-05-05 05:12:05,4,5045
2,Manually setting session ID in Express,angularexpress app trying implement kind restful auth express app passport standard usernamepass login redis session successful login return session id angular sends every request header problem dont know make express make use session id tried writing reqsessionid middleware success use header query string way send session id along,"[javascript, angularjs, node.js, express, passport.js]",20060800,4,3606,1,2013-11-19 00:27:34,2015-01-08 16:58:59,2,415
3,Include Google Maps API Key in open source project?,okay put google map api key source code publish others could take misuse dont want every developer user get api key type somewhere owner key responsible create new google accoun

In [105]:
df.shape

(50000, 11)

In [106]:
# Initialiser le vectoriseur de mots
vectorizer = CountVectorizer()

# Appliquer le vectoriseur aux questions
X = vectorizer.fit_transform(df['Body'])

# Obtenir la liste des mots (features)
features = vectorizer.get_feature_names()

# Afficher la matrice des fonctionnalités
print("Matrice des fonctionnalités :")
print(X)

# Afficher la liste des mots (features)
print("Liste des mots (features) :")
print(features)

C:\Users\omira\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Matrice des fonctionnalités :
  (0, 679278)	1
  (0, 315440)	2
  (0, 685884)	2
  (0, 147713)	2
  (0, 571309)	1
  (0, 342730)	1
  (0, 205513)	1
  (0, 407212)	2
  (0, 486898)	1
  (0, 446112)	2
  (0, 600211)	1
  (0, 254424)	1
  (0, 132309)	1
  (0, 286158)	1
  (0, 226708)	2
  (0, 428084)	1
  (0, 67884)	1
  (0, 685559)	2
  (0, 79689)	3
  (0, 552052)	1
  (0, 359142)	1
  (0, 212966)	1
  (0, 659322)	1
  (0, 342733)	1
  (0, 481001)	1
  :	:
  (49998, 502002)	1
  (49998, 282480)	1
  (49998, 436506)	1
  (49998, 413850)	1
  (49998, 329523)	1
  (49998, 94909)	2
  (49998, 685179)	1
  (49998, 401305)	1
  (49998, 136701)	1
  (49998, 248269)	1
  (49998, 149742)	1
  (49999, 450108)	1
  (49999, 338469)	1
  (49999, 636936)	1
  (49999, 357622)	1
  (49999, 153477)	1
  (49999, 180651)	1
  (49999, 90726)	1
  (49999, 546187)	1
  (49999, 90640)	3
  (49999, 521712)	1
  (49999, 603908)	2
  (49999, 187837)	1
  (49999, 264665)	1
  (49999, 271172)	1
Liste des mots (features) :


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

